# Given query of length 200 and reference of big length
## How good is the classification?

In [1]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook

import Config
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


In [3]:
def perm_inv(p):
    s = np.empty_like(p)
    s[p] = np.arange(p.size)
    return s

In [4]:
def perm_exp(p, n):
    s = np.arange(len(p))
    for i in range(n):
        s = s[p]
    return s

In [5]:
def preprocess_bind(basis, c_len, perm):
    mapping = dict()
    if c_len == 0:
        mapping[""] = -np.ones(len(basis[0]))
        return mapping
    suf_mapping = preprocess_bind(basis, c_len - 1, perm)
    for suffix in suf_mapping.keys():
        suf_enc = suf_mapping[suffix]
        perm_enc = suf_enc[perm]
        for i in range(4):
            currkey = str(i) + suffix
            mapping[currkey] = - (basis[i] * perm_enc)
    return mapping

In [6]:
def bind(chunk, basis, perm = None):
    D = len(basis[0])
    if perm is None:
        perm = np.arange(D)
    result = -np.ones(D)
    for c in reversed(chunk):
        result = -(result[perm] * basis[int(c)])
    return result

In [7]:
def bundling(vecs, perm = None):
    D = len(vecs[0])
    if perm is None:
        perm = np.arange(D)
    result = np.zeros(D)
    for vec in reversed(vecs):
        result = result[perm] + vec
    return result

In [8]:
def binarize(X):
    return np.where(X >= 0, 1, -1)

In [9]:
def chunk_string(string, c_len, overlap = False):
    if not overlap:
        return [string[c_len * i: c_len * (i+1)] for i in range(len(string)//c_len)]
    else:
        return [string[i:i + c_len] for i in range(len(string) - c_len + 1)]

In [10]:
# String: string of arbitrary length for encoding
# c_len: chunk length to bind together
# Note: non-overlapping by default
# s_len: substring length 
# perm1: permutation used for binding, not needed given a complete mapping 
# perm2: perm used for bundling among chunks
# perm3: perm used for bunding among substrings
def encode_DNA(mapping, string, c_len, s_len, perm2, basis = None, perm1 = None, overlap = False, debug = False):
    
    global D
    
    chunks = chunk_string(string, c_len, overlap)
    cps = s_len // c_len # Chunk per substring
    enc_strs = []
    enc_chunks = []
    
    if not overlap: 
        
        n_str =  len(chunks) - cps + 1

        # Prep prefix. Head chunk is the chunk to delete next. Initialized to 0
        if mapping is None:
            enc_chunks = [bind(chunks[i], basis, perm1) for i in range(cps - 1)]
            #print(enc_chunks)
        else:
            enc_chunks = [mapping[chunks[i]] for i in range(cps - 1)]
            #print(enc_chunks)
        #print(enc_chunks)

        head_enc_chunk = np.zeros(D)
        if len(enc_chunks) != 0:
            prefix = bundling(enc_chunks, perm2)
            prefix = prefix[perm2]
        else:
            prefix = np.zeros(D)

        perm2_inv = perm_inv(perm2)
        perm2_fin = perm_exp(perm2, cps - 1)

        for i in range(n_str):
            prefix -= head_enc_chunk
            prefix = prefix[perm2_inv]

            if mapping is None:
                curr_enc_chunk = bind(chunks[cps + i - 1], basis, perm1)
            else:
                curr_enc_chunk = mapping[chunks[cps + i - 1]] # Find the next final chunk

            curr_enc_chunk = curr_enc_chunk[perm2_fin]
            prefix += curr_enc_chunk

            if debug:
                print("Current encoded substring:", prefix)
            enc_strs.append(np.copy(prefix))

            if mapping is None:
                head_enc_chunk = bind(chunks[i], basis, perm1)
            else:
                head_enc_chunk = mapping[chunks[i]] # Retrieve the next first chunk

        enc_string = bundling(enc_strs)
        return enc_string
    else:
        pass

In [11]:
def encode_data(mapping, strings, c_len, s_len, perm2, basis = None, perm1 = None):
    encs = []
    for string in tqdm_notebook(strings, desc='epochs'):
        encs.append(encode_DNA(mapping, string, c_len, s_len, perm2,  basis, perm1))
    return encs
    

In [12]:
def evaluate(enc_q, enc_rs):
    vals = np.asarray([np.dot(enc_q, enc_r) for enc_r in enc_rs])
    order = np.asarray(list(reversed(np.argsort(vals))))
    return order, vals

In [13]:
def evaluate_unaligned(enc_q, enc_rs, c_len, perm):
    def opt_dot(enc_q, enc_r, c_len, perm):
        best_val = None
        curr_q = enc_q
        for i in range(c_len):
            curr_val = np.dot(curr_q, enc_r)
            if best_val is None or curr_val > best_val:
                best_val = curr_val
            curr_q = curr_q[perm]  
        return best_val
    vals = np.asarray([opt_dot(enc_q, enc_r, c_len, perm) for enc_r in enc_rs])
    order = np.asarray(list(reversed(np.argsort(vals))))
    return order, vals

In [14]:
q_file = open("query.txt", "r")
queries = q_file.read().split("\n")
queries = queries[:-1]
q_file.close()

r_file = open("ref.txt", "r")
refs = r_file.read().split("\n")
refs = refs[:-1]
r_file.close()

l_file = open("lab.txt", "r")
labs = l_file.read().split("\n")
labs = labs[:-1]
l_file.close()

print(len(queries))
print(len(refs))
print(len(labs))

100
100
100


In [15]:
D = 10000
basis = np.random.randint(0, 2, (4,D))  * 2 - 1 
print(np.sum(basis, axis = 1))

perm1 = np.random.permutation(D)
perm2 = np.random.permutation(D)

perm3 = np.arange(D-1)
perm3 = np.append([D-1], perm3)

print(perm1)
print(perm2)
print(perm3)

[  42    0   50 -160]
[2594 3494 1793 ... 6221 6002 2257]
[8878 9871 9884 ... 3807 1529 2241]
[9999    0    1 ... 9996 9997 9998]


In [16]:
c_len = 4
s_len = len(queries[0])
print(c_len, s_len)

4 256


In [17]:
# If c_len is big, no mapping 
# enc_qs = encode_data(None, queries, c_len, s_len, perm2, basis, perm1)
# enc_rs = encode_data(None, refs, c_len, s_len, perm2, basis, perm1)

In [18]:
thresh = math.log((len(queries)*len(queries[0])+len(refs)*len(refs[0]))//c_len, 4) - c_len  

if thresh >= 0:
    # If c_len is small enough
    print("small chunk length, preprocessing chunks")
    mapping = preprocess_bind(basis, c_len, perm2)
    #print(mapping)
    enc_qs = encode_data(mapping, queries, c_len, s_len, perm_exp(perm2, c_len-1))
    enc_rs = encode_data(mapping, refs, c_len, s_len, perm_exp(perm2, c_len-1))
else: 
    enc_qs = encode_data(None, queries, c_len, s_len, perm2, basis, perm2)
    enc_rs = encode_data(None, refs, c_len, s_len, perm2, basis, perm2)

small chunk length, preprocessing chunks


c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
correct = 0
count = 0
correct_unaligned = 0

for i in range(0, len(enc_qs)):
    print("%dth substring:################"%(i))
    order, vals = evaluate_unaligned(enc_qs[i], enc_rs, c_len, perm2)
    
    # Handle intended matching
    if i < len(labs):
        j = np.where(order == i)[0]
        print("\t Match injected at %dth choice %d with value %d at index %s"%
              (j+1, order[j], vals[order[j]], labs[i]))
        if j == 0 and int(labs[i]) % c_len == 0:
            correct += 1
        if j == 0:
            correct_unaligned += 1
        if int(labs[i]) % c_len == 0:
            count += 1

    # Hanlde unintended perfect matching
    for j in range(len(order)):
        pos = refs[order[j]].find(queries[i])
        if pos != -1:
            print("\t Match found at %dth choice %d with value %d at index %d"%
                  (j+1, order[j], vals[order[j]], pos))
                
    print("\t\t Top choice %d with value %d"%(order[0], vals[order[0]]))
    print("\t\t 2nd choice %d with value %d"%(order[1], vals[order[1]])) 
                
print(correct, count, correct_unaligned)

0th substring:################
	 Match injected at 2th choice 0 with value 2347284 at index 660
		 Top choice 17 with value 2353664
		 2nd choice 0 with value 2347284
1th substring:################
	 Match injected at 1th choice 1 with value 2800224 at index 836
		 Top choice 1 with value 2800224
		 2nd choice 93 with value 2466068
2th substring:################
	 Match injected at 1th choice 2 with value 2232208 at index 124
		 Top choice 2 with value 2232208
		 2nd choice 9 with value 2152600
3th substring:################
	 Match injected at 1th choice 3 with value 2224700 at index 464
		 Top choice 3 with value 2224700
		 2nd choice 26 with value 1930436
4th substring:################
	 Match injected at 1th choice 4 with value 2291380 at index 776
		 Top choice 4 with value 2291380
		 2nd choice 17 with value 2234428
5th substring:################
	 Match injected at 1th choice 5 with value 2583736 at index 80
		 Top choice 5 with value 2583736
		 2nd choice 32 with value 2334128


	 Match injected at 1th choice 51 with value 2440384 at index 596
		 Top choice 51 with value 2440384
		 2nd choice 12 with value 2060816
52th substring:################
	 Match injected at 1th choice 52 with value 2715868 at index 840
		 Top choice 52 with value 2715868
		 2nd choice 34 with value 2174364
53th substring:################
	 Match injected at 1th choice 53 with value 2441140 at index 556
		 Top choice 53 with value 2441140
		 2nd choice 97 with value 2238776
54th substring:################
	 Match injected at 1th choice 54 with value 2484836 at index 164
		 Top choice 54 with value 2484836
		 2nd choice 25 with value 2005696
55th substring:################
	 Match injected at 1th choice 55 with value 2228400 at index 892
		 Top choice 55 with value 2228400
		 2nd choice 18 with value 1871932
56th substring:################
	 Match injected at 1th choice 56 with value 2521312 at index 256
		 Top choice 56 with value 2521312
		 2nd choice 6 with value 2411400
57th substrin

In [20]:
print(labs)

['660', '836', '124', '464', '776', '80', '20', '460', '260', '692', '604', '832', '628', '184', '788', '468', '720', '396', '848', '916', '996', '984', '308', '196', '184', '648', '436', '668', '852', '192', '132', '984', '680', '496', '168', '24', '964', '688', '324', '820', '64', '516', '268', '624', '920', '608', '100', '792', '260', '1016', '568', '596', '840', '556', '164', '892', '256', '736', '632', '68', '140', '360', '996', '576', '564', '632', '848', '316', '828', '840', '60', '20', '52', '784', '948', '420', '660', '428', '256', '124', '332', '640', '476', '848', '320', '988', '932', '992', '1024', '912', '140', '952', '144', '740', '480', '384', '128', '128', '984', '480']


In [21]:
c_len = 2
s_len = 4
string = "0123"

print(basis)
mapping = preprocess_bind(basis, c_len, perm1)
print(mapping)
print(encode_DNA(mapping, string, c_len, s_len, perm2, perm3, True))


[[-1  1  1 ...  1  1 -1]
 [-1 -1 -1 ... -1  1  1]
 [ 1 -1 -1 ... -1 -1 -1]
 [-1  1  1 ...  1 -1 -1]]
{'00': array([ 1., -1., -1., ...,  1., -1., -1.]), '10': array([ 1.,  1.,  1., ..., -1., -1.,  1.]), '20': array([-1.,  1.,  1., ..., -1.,  1., -1.]), '30': array([ 1., -1., -1., ...,  1.,  1., -1.]), '01': array([ 1., -1.,  1., ...,  1., -1.,  1.]), '11': array([ 1.,  1., -1., ..., -1., -1., -1.]), '21': array([-1.,  1., -1., ..., -1.,  1.,  1.]), '31': array([ 1., -1.,  1., ...,  1.,  1.,  1.]), '02': array([ 1., -1.,  1., ...,  1.,  1., -1.]), '12': array([ 1.,  1., -1., ..., -1.,  1.,  1.]), '22': array([-1.,  1., -1., ..., -1., -1., -1.]), '32': array([ 1., -1.,  1., ...,  1., -1., -1.]), '03': array([-1.,  1.,  1., ...,  1., -1.,  1.]), '13': array([-1., -1., -1., ..., -1., -1., -1.]), '23': array([ 1., -1., -1., ..., -1.,  1.,  1.]), '33': array([-1.,  1.,  1., ...,  1.,  1.,  1.])}
[2. 0. 2. ... 2. 0. 2.]


In [22]:
np.set_printoptions(threshold=np.inf)